1) Написать программу, которая собирает входящие письма из своего или тестового почтового ящика и сложить данные о письмах в базу данных (от кого, дата отправки, тема письма, текст письма полный)

2) Написать программу, которая собирает «Хиты продаж» с сайта техники mvideo и складывает данные в БД. Магазины можно выбрать свои. Главный критерий выбора: динамически загружаемые товары!

### Задание 1

In [3]:
from selenium import webdriver
from selenium.webdriver.common.by import By  
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select  
from selenium.webdriver.common.action_chains import ActionChains
import time
import pandas as pd
from pymongo import MongoClient
from selenium.webdriver.chrome.options import Options

In [31]:
from_list=[]
date_list=[]
subject_list=[]
text_list=[]
driver = webdriver.Chrome()   
driver.get('https://mail.ru')
assert "Mail.ru" in driver.title

#Заполняем поля для ввода
login = driver.find_element_by_id("mailbox:login")
login.send_keys('study.ai_172@mail.ru')
login.submit()
#Кнопка введите пароль
btn_pass_ent = driver.find_element_by_id("mailbox:submit")
btn_pass_ent.click()
password = driver.find_element_by_id("mailbox:password")
#password = WebDriverWait(driver,5).until(EC.presence_of_element_located((By.ID, 'mailbox:password')))     
password.send_keys('Password172')
password.submit()
time.sleep(5)
#news = driver.find_element_by_class_name("mt-t_newsletters")
action = ActionChains(driver);
#action.move_to_element(news).perform()
#news.click()
#driver.implicitly_wait(3)
#content = driver.find_elements_by_class_name('ll-sj__normal')
#for elem in content:
#        from_list.append(elem.text)
all_ = driver.find_element_by_class_name("ico_16-burger")
action.move_to_element(all_).perform()
all_.click()
driver.implicitly_wait(1)
#folders = WebDriverWait(driver,3).until(EC.presence_of_element_located((By.CLASS_NAME, 'nav__folder-name__txt')))  
folders = driver.find_elements_by_class_name("nav__folder-name__txt")
folder = folders[-5]
action.move_to_element(folder).perform()
folder.click()
time.sleep(2)
i = 0
while True:
    try:
        content = driver.find_elements_by_class_name('llc__container')
        action.move_to_element(content[i])
        content[i].click()
        from_ = driver.find_element_by_class_name('letter__contact-item')
        from_list.append(from_.text)
        date = driver.find_element_by_class_name('letter__date')
        date_list.append(date.text)
        subject = driver.find_element_by_class_name('thread__subject')
        subject_list.append(subject.text)
        try :
            text = driver.find_element_by_class_name('letter__body')
            text_list.append(text.text)
        except:
            text_list.append(None)
        time.sleep(1)
        back = driver.find_element_by_class_name("ico_16-arrow-back")
        back.click()
        time.sleep(1)
        i+=1
    except IndexError:
        break
    except Exception as e:
        print(e)
        pass

Message: element click intercepted: Element <div class="llc__container">...</div> is not clickable at point (529, 900). Other element would receive the click: <div class="scrollable g-scrollable scrollable_bright scrollable_footer">...</div>
  (Session info: chrome=77.0.3865.90)

Message: element click intercepted: Element <div class="llc__container">...</div> is not clickable at point (529, 900). Other element would receive the click: <div class="scrollable g-scrollable scrollable_bright scrollable_footer">...</div>
  (Session info: chrome=77.0.3865.90)

Message: element click intercepted: Element <div class="llc__container">...</div> is not clickable at point (529, 900). Other element would receive the click: <div class="scrollable g-scrollable scrollable_bright scrollable_footer">...</div>
  (Session info: chrome=77.0.3865.90)

Message: element click intercepted: Element <div class="llc__container">...</div> is not clickable at point (529, 900). Other element would receive the click

In [32]:
len(text_list)

24

In [34]:
s_from = pd.Series(from_list)
s_date = pd.Series(date_list)
s_subject = pd.Series(subject_list)
s_text = pd.Series(text_list)
df = pd.DataFrame()
df['from'] = s_from
df['date'] = s_date
df['subject'] = s_subject
df['text'] = s_text
df.to_csv('letters.csv',index=False,header=True, encoding='utf-8')

In [35]:
df.head()

,from,date,subject,text
0,Tproger Live,"Сегодня, 20:18",Вечерний обзор IT-новостей 2 октября,Налоги в Сети\nБританская компания Comparitech...
1,Tproger Live,"Вчера, 21:02",Вечерний обзор IT-новостей 1 октября,Яндекс.Облако\nСегодня утром прошла Yandex Sca...
2,GeekBrains,"Вчера, 11:27",<Открытый урок> Верстаем сайт по макету,"Если в письме нет картинок, жмите сюда..."
3,Tproger Live,"30 сентября, 20:26",Вечерний обзор IT-новостей 30 сентября,"Бэкдоры для соседа\nКакой-то инсайдер, знакомы..."
4,Tproger Live,"27 сентября, 20:44",Вечерний обзор IT-новостей 27 сентября,"Интернет\nChrome, Firefox и Cloudflare начинаю..."


In [36]:
client = MongoClient('localhost',27017)
db = client['letters']

In [37]:
data = df.to_dict(orient='records')
db = db.letters
db.insert_many(data)

### Задание 2

In [23]:
g = []
chrome_options = Options()
chrome_options.add_argument("--window-size=1920,1080")
driver = webdriver.Chrome(chrome_options=chrome_options)
driver.get('https://www.mvideo.ru')
assert "М.Видео" in driver.title
time.sleep(3)
exit = False
while not exit:
    try:
        goods = driver.find_elements_by_class_name("sel-product-tile-title")
        for name in goods[0:4]:
                if len(g) and name.text in g:
                    exit = True
                g.append(name.text)
        #button_next = driver.find_elements_by_class_name("sel-hits-button-next")
        #button_next = driver.find_element_by_css_selector(".sel-hits-button-next")
        button_next = driver.find_elements_by_xpath('//*[@class="accessories-carousel-wrapper"]/a[2]')
        action = ActionChains(driver);
        action.move_to_element(button_next[1])
        button_next[1].click()
        time.sleep(2)
    except Exception as e:
        print(e)
        break

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: DeprecationWarning: use options instead of chrome_options
  after removing the cwd from sys.path.


In [24]:
g

['Игровая консоль Xbox One Microsoft S 1TB белая с двумя геймпадами',
 'Телевизор Samsung UE43NU7097U',
 'Смартфон Apple iPhone XR 64GB Black (MRY42RU/A)',
 'Стиральная машина узкая Haier HW60-12829A',
 'Смартфон Apple iPhone 7 Plus 32Gb Rose Gold (MNQQ2RU/A)',
 'Электромясорубка Bosch ProPower MFW66020',
 'Смартфон Honor 10I 128Gb Midnight black(HRY-LX1T)',
 'Холодильник Hotpoint-Ariston HFP 5200 W',
 'Ноутбук Lenovo IdeaPad 330-15IKB (81DE02VRRU)',
 'Конвектор VITEK VT-2174 W',
 'Ноутбук Lenovo IdeaPad S145-15IWL (81MV00HHRU)',
 "Утюг Bosch sensixx'x DA50 TDA503011P",
 'Беспроводная акустика iHome iBT372 Black',
 'Мультиварка Tefal Advanced RK812832',
 'Наушники Marshall Major III Bluetooth Black',
 'Пылесос ручной (handstick) Tefal Air Force 160 red TY7233WO',
 'Беспроводная акустика Harman/Kardon Go + Play Mini Black (HKGOPLAYMINIBLKEU)',
 'Фен Rowenta Studio Dry CV5361F0',
 'Струйное МФУ HP Ink Tank 315',
 'Электрочайник Moulinex BY730132',
 'Беспроводная акустика Harman/Kardon Go

In [26]:
s_name = pd.Series(g)
df = pd.DataFrame()
df['name'] = s_name
df.to_csv('bestsellers.csv',index=False,header=True, encoding='utf-8')
df.head()

,name
0,Игровая консоль Xbox One Microsoft S 1TB белая...
1,Телевизор Samsung UE43NU7097U
2,Смартфон Apple iPhone XR 64GB Black (MRY42RU/A)
3,Стиральная машина узкая Haier HW60-12829A
4,Смартфон Apple iPhone 7 Plus 32Gb Rose Gold (M...


In [27]:
client = MongoClient('localhost',27017)
db = client['bestsellers']

In [28]:
data = df.to_dict(orient='records')
db = db.letters
db.insert_many(data)